In [1]:
from mvpa2.tutorial_suite import *
import os

## Load fmri data

In [6]:
# directory that contains the data files
home_dir = os.getcwd()
data_path = os.path.join(home_dir, 'data', 'haxby2001')
# load data from: Haxby et al. (2001): Faces and Objects in Ventral Temporal Cortex (fMRI)
# a study were participants passively watched gray scale images of eight object categories
#in a block-design experiment (12 runs of subj0001)
attributes_fname = os.path.join(data_path, 'sub001',
                         'BOLD', 'task001_run001', 'attributes.txt')
#For a classification analysis we also need to associate each sample with 
#a corresponding experimental condition, i.e. a class label, also sometimes called target value

#for a cross-validation procedure we also need to partition the full dataset into, presumably, independent chunks

#target value for an fMRI volume sample = experiment condition present/active while the volume was acquired

#chunks of independent data =  fMRI volumes are assumed to be independent
#Ideally, the experiment is split into several acquisition sessions, 
#where the sessions define the corresponding data chunks.
attributes = SampleAttributes(attributes_fname)
attributes

{'chunks': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'targets': ['rest',
  'rest',
  'rest',
  'rest',
  'rest',
  'rest',
  'scissors',
  'scissors',
  'scissors',
  'scissors',
  'scissors',
 

In [60]:
print(len(attributes.targets))
print(len(attributes.chunks))
print(np.unique(attributes.targets))

121
121
['bottle' 'cat' 'chair' 'face' 'house' 'rest' 'scissors' 'scrambledpix'
 'shoe']


SampleAttributes allows us to load this type of file, and access its content:
- 121 targets(experimental condition) and chunk values, one for each volume. 
- 9 different conditions and all samples are associated with the same chunk.
The attributes file for a different scan/run would increment the chunk value.

In [61]:
#load fmri data
#data is stored as a NIfTI file that has all volumes of one experiment concatenated into a single file.

bold_fname = os.path.join(data_path,
                          'sub001', 'BOLD', 'task001_run001', 'bold.nii.gz')
mask_fname = os.path.join(data_path,
                          'sub001', 'masks', 'orig', 'vt.nii.gz')
fds = fmri_dataset(samples=bold_fname,
                   targets=attributes.targets, chunks=attributes.chunks,
                    mask=mask_fname)

print(fds.nfeatures) #number of features or voxels in volume
print(len(fds)) #number of samples or volumes of nfti file
fds.shape #two-dimensional dataset

577
121


(121, 577)

In [62]:
# In case you want to access specific information about volume or voxel
# get volume index in time-series
print(fds.sa.time_indices[:5])
# get acuisition time
print(fds.sa.time_coords[:5])
# get voxel index for each feature
print(fds.fa.voxel_indices[:5])
# get input volumes dimensions
print(fds.a.voxel_eldim)
# get voxel dimensionality 
print(fds.a.voxel_dim)

[0 1 2 3 4]
[ 0.   2.5  5.   7.5 10. ]
[[ 6 23 24]
 [ 7 18 25]
 [ 7 18 26]
 [ 7 18 27]
 [ 7 19 25]]
(3.5, 3.75, 3.75)
(40, 64, 64)


### Acess big data sets

In [63]:
#if you have multiple data sets, use support for dataset
# access it via OpenFMRIDataset
dhandle = OpenFMRIDataset(data_path)
dhandle.get_subj_ids()

[1]

In [64]:
#if you want to access details about experimental design
model = 1
subj = 1
run = 1
events = dhandle.get_bold_run_model(model, subj, run)
for ev in events[:2]:
     print ev

{'task': 1, 'run': 1, 'onset_idx': 0, 'conset_idx': 0, 'onset': 15.0, 'intensity': 1, 'duration': 22.5, 'condition': 'scissors'}
{'task': 1, 'run': 1, 'onset_idx': 1, 'conset_idx': 0, 'onset': 52.5, 'intensity': 1, 'duration': 22.5, 'condition': 'face'}


In [65]:
#if you want to match stim onsets to data samples
targets = events2sample_attr(events, fds.sa.time_coords,
                              noinfolabel='rest', onset_shift=0.0) #in case timing is off)
print (np.unique([attributes.targets[i] == targets[i] for i in range(len(targets))])) #check if len is matching
stim_samples_dict = dict(zip(targets, fds))
stim_samples_dict

[ True]


{'bottle': Dataset(array([[1197,  792, 1238, 1380, 1063, 1465, 1431, 1604, 1355, 1823, 1013,
         1661, 1735,  874, 1631, 1624, 1289, 1394, 1533, 1661, 1501, 1654,
         1550, 1571, 1770, 1606,  752, 1468, 1652, 1586, 1214, 1734, 1764,
         1417, 1847, 1768, 1302, 1719, 1645, 1888, 1170, 1537,  811, 1601,
         1497, 1364, 1579, 1701,  889, 1611, 1542, 1677, 1665,  930, 1670,
         1737, 1784, 1740, 1691,  782, 1801, 1771, 1702, 1645,  978, 1866,
         1765, 1054, 1739, 1989, 1070, 1332, 1489, 1908, 1509, 1359, 1610,
         1571, 1466, 1487, 1697, 1155, 1702, 1506, 1696, 1872, 1747,  858,
         1757, 1641, 1800, 1757,  831, 1833, 1656, 1830, 1812, 1808,  809,
         2040, 1888, 1765, 1061, 1944, 1255, 1996, 1897, 1338, 1974, 2198,
         1363, 2096, 2017, 1505, 1729, 1603, 1132, 1748, 1662, 1674, 1065,
         1853, 1728, 1671, 1758, 1258, 1821, 1811, 1741, 1624, 1868, 1795,
         1788, 1748, 1890, 1632, 1781, 2033, 1866, 1758, 1869, 2130, 2159,
       

### Access data with multiple session

In [66]:
task = 1   # object viewing task
model = 1  # image stimulus category model
subj = 1
run_datasets = []
for run_id in dhandle.get_task_bold_run_ids(task)[subj]:
     # load design info for this run
    run_events = dhandle.get_bold_run_model(model, subj, run_id)
     # load BOLD data for this run (with masking); add 0-based chunk ID
    run_ds = dhandle.get_bold_run_dataset(subj, task, run_id,
                                           chunks=run_id -1,
                                           mask=mask_fname)
     # convert event info into a sample attribute and assign as 'targets'
    run_ds.sa['targets'] = events2sample_attr(
                 run_events, run_ds.sa.time_coords, noinfolabel='rest')
     # additional time series preprocessing can go here
    run_datasets.append(run_ds)
# a=0 indicates that the dataset attributes of the first run should be used
# for the merged dataset
fds = vstack(run_datasets, a=0)
#fds

### Store temporary data and recall it

In [67]:
import tempfile, shutil

In [68]:
#save data 
h5save(os.path.join(home_dir, 'mydataset.hdf5'), fds, compression=9)
#recall data
loaded = h5load(os.path.join(home_dir, 'mydataset.hdf5'))

## Simpel Preprocessing

### Detrending

In [69]:
#if you want to remove polynomial trends from data, create mapper that will do chunkwise linear detrending
# Chunk-wise detrending is desirable, since our data stems from different runs,
# and the assumption of a continuous linear trend across all runs is not appropriate. 

#IMPORTANT: if you plan on modelling your data with haemodynamic response functions 
#in a general linear model (like it is shown in Working with OpenFMRI.org data with NiPy),
#polynomial detrending can be done simultaneously as part of the modeling.

detrender = PolyDetrendMapper(polyord=1, chunks_attr='chunks')
# if you do not want to make copy with detrended data, use poly_detrend
#detrender = poly_detrend(polyord=1, chunks_attr='chunks')

detrended_fds = fds.get_mapped(detrender)
print (detrended_fds.a.mapper)

<Chain: <Flatten>-<StaticFeatureSelection>-<PolyDetrend: ord=1>>


### Normalization

In [70]:
# if we want to get rid off inhomogeneous voxel intensities that can be a problem for some machine learning algorithms
# perform a feature-wise, chunk-wise Z-scoring of the data
# we are not going to perform a very simple Z-scoring removing the global mean, but use the rest condition samples 
#of the dataset to estimate mean and standard deviation. Scaling dataset features using these parameters 
#yields a score corresponding to the per time-point voxel intensity difference from the rest average.

zscorer = ZScoreMapper(param_est=('targets', ['rest'])) #chunkwise Z-scoring of data in "rest" condition(target)
# if you do not want to make copy with normalized data, use zscore()
#zscorer = zscore(param_est=('targets', ['rest']))

### Remove condition from dataset

In [71]:
# if you do not need rest condition anymore, remove from data set
fds = fds[fds.sa.targets != 'rest']

## Computing Patterns Of Activation

In [76]:
# NORAMLLY: you perform a GLM-analysis of odd vs. even runs of the data respectively #
#then use the corresponding contrast statistics (stimulus category vs. rest) as classifier input

# Also, commonly the data is split by leave-one out crossvalidation, where one chunk of the data
# is testing data and the other training data. Iterate till every chunk was testing data
# average
run_averager = mean_group_sample(['targets', 'chunks'])
fds_leave_one_out = fds.get_mapped(run_averager)
fds_leave_one_out.shape # 12 runs* 9-rest=8 conditions = 96 samples

(96, 577)

In [75]:
# HERE: only compute mean samples per condition for both odd vs. even runs independently
#1: add a new sample attribute to assign a corresponding label to each sample in the dataset that 
#indicates which of both run-types it belongs to

rnames = {0: 'even', 1: 'odd'}
fds.sa['runtype'] = [rnames[c % 2] for c in fds.sa.chunks]

#2: mean_group_sample() takes a list of sample attributes, determines all possible combinations of its unique values,
#selects dataset samples corresponding to these combinations

averager = mean_group_sample(['targets', 'runtype'])
fds_split = fds.get_mapped(averager)
print(fds.shape)
fds_split.sa.targets

(864, 577)


array(['bottle', 'cat', 'chair', 'face', 'house', 'scissors',
       'scrambledpix', 'shoe', 'bottle', 'cat', 'chair', 'face', 'house',
       'scissors', 'scrambledpix', 'shoe'], dtype='|S12')

# Classifiers

In [45]:
# HERE: easy access to already preprocessed data
# Delete for new data set!
fds = get_haxby2001_data()
#if does not work change path in tutorial_suite.py "def get_haxby2001_data" to home_dir

# Haxby study employed a so-called 1-nearest-neighbor classifier, using correlation as a distance measure. 
#k-Nearest-Neighbor classifier performs classification based on the similarity of a sample with respect to each sample 
#in a training dataset. The value of k specifies the number of neighbors to derive a prediction, dfx sets the distance 
#measure that determines the neighbors, and voting selects a strategy to choose a single label from the set of targets 
#assigned to these neighbors

clf = kNN(k=1, dfx=one_minus_correlation, voting='majority')

In [46]:
#train classifier 
clf.train(ds)

In [47]:
#use trained classifier to classify unlabeled samples
#test accuracy by comparing predictions and initial target labels
predictions = clf.predict(fds.samples)
np.mean(predictions == fds.sa.targets)

1.0

### Label unlabeld new data

In [48]:
#IMPORTANT: if single dataset, split it in odd versus even numbered trials
fds.sa.runtype
#Split rows in datasets(=samples) by odd or even
ds_split1 = fds[fds.sa.runtype == 'odd']
ds_split2 = fds[fds.sa.runtype == 'even']
print(len(ds_split1),len(ds_split2))

(8, 8)


In [49]:
#compute the mismatch error between the classifier predictions and the target values stored in our dataset
clf.set_postproc(BinaryFxNode(mean_mismatch_error, 'targets'))
clf.train(ds_split2)
error = clf(ds_split1)
print(np.asscalar(error)) #lower values reflect more accurate classification

0.125


### Cross-validation

In [53]:
# to automatically split up dataset and cross-validate
# disable post-processing again
clf.set_postproc(None)
# dataset generator
hpart = HalfPartitioner(attr='runtype')
# complete cross-validation facility
cv = CrossValidation(clf, hpart) #trains and tests the classifier repeatedly
cv_results = cv(fds)
print(np.mean(cv_results))
# cross-validated results returned as additional dataset, which has 
# one sample per fold and a single feature with the computed transfer-error per fold
cv_results.samples

0.0625


array([[0.   ],
       [0.125]])

### Other classifiers : Support Vcetor Machines

In [56]:
clf = LinearCSVMC()
cvte = CrossValidation(clf, HalfPartitioner(attr='runtype'))
cv_results = cvte(fds)
np.mean(cv_results)

0.1875

### Compute Accuracy 

In [57]:
# Basically takes 1 minus last result 
cvte = CrossValidation(clf, HalfPartitioner(attr='runtype'),
                       errorfx=lambda p, t: np.mean(p == t))
cv_results = cvte(fds)
np.mean(cv_results)

0.8125

### Compute Accuracy for Leave-One-Run-Out

In [77]:
# if you want to select samples one at a time, use NFoldPartitioner
cvte = CrossValidation(clf, NFoldPartitioner(),
                       errorfx=lambda p, t: np.mean(p == t))
cv_results = cvte(fds_leave_one_out)
np.mean(cv_results)

0.2708333333333333

In [78]:
cv_results.samples

array([[0.25 ],
       [0.25 ],
       [0.5  ],
       [0.625],
       [0.25 ],
       [0.375],
       [0.125],
       [0.25 ],
       [0.125],
       [0.25 ],
       [0.125],
       [0.125]])

 Digging deeper we want to know about missclassification --
 For example: missclassification of half of the samples of each condition as well as missclass. of all samples 
 of half of the conditions(8/2=4 in our case) give the same prediction error. b
 

In [79]:
# enable_ca display all cross validations in a confusion table 
cvte = CrossValidation(clf, NFoldPartitioner(),
                       errorfx=lambda p, t: np.mean(p == t),
                      enable_ca=['stats'])
cv_results = cvte(fds_leave_one_out)
print(cvte.ca.stats.as_string(description=True))

----------.
predictions\targets     bottle         cat          chair          face         house        scissors    scrambledpix      shoe
            `------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------ P'  N'   FP   FN   PPV  NPV  TPR  SPC  FDR  MCC   F1
       bottle             2             1             1             1             0             2             0             3       10  34    8   10   0.2 0.71 0.17 0.75  0.8 -0.05 0.18
        cat               2             1             1             1             2             2             2             3       14  36   13   11  0.07 0.69 0.08 0.66 0.93 -0.17 0.08
       chair              1             3             2             1             2             2             1             1       13  34   11   10  0.15 0.71 0.17 0.69 0.85 -0.09 0.16
        face              1             4             2             6             0             2             3      

In [80]:
# if you want to show pure matrix form
cvte.ca.stats.matrix

array([[2, 1, 1, 1, 0, 2, 0, 3],
       [2, 1, 1, 1, 2, 2, 2, 3],
       [1, 3, 2, 1, 2, 2, 1, 1],
       [1, 4, 2, 6, 0, 2, 3, 1],
       [0, 1, 1, 0, 6, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [3, 1, 3, 3, 1, 1, 5, 0],
       [3, 1, 2, 0, 1, 1, 1, 3]])

# SEARCHLIGHT

So far we have no clue about where in the brain (or our chosen ROIs) our signal of interest is located.
What we want is to estimate a score for each feature(voxel) that indicates how important that particular feature is in the context of a certain classification task

In [87]:
# get data, agaain
ds = get_haxby2001_data(roi='vt')
ds.shape
print("nVolumes:",len(ds))
print("nVoxels:",ds.nfeatures)

('nVolumes:', 16)
('nVoxels:', 577)


### Measurements for score of each feature

In [86]:
# ANOVA F-Score
# F-score per each feature, yielding scores that would tell us which features vary significantly 
# between any of the categories in our dataset
aov = OneWayAnova()
f = aov(ds)


577